# Deep Learning Model( For Regression ):
## CNN, CNN + RNN

#### Case 1 : (CNN)
- We take single image --> predict an angle
- So here we are not using sequential information( not taking sequence of images )
- We can have ConvNet based regression( just the loss will change ).

#### Case 2 : (CNN + RNN)
- Images + Seq      ----->   Angle + Sequence
- input(Video)    ------>          Output(Seq of real numbers)
- image ---> CNN---> [A vector]---> RNN(lstm)(many to many)---> yi

- We will build CASE 1 model using ConvNet because we have limited computational resources for CASE 2 using CNN + RNN, and such models(case1) are called end to end models.

- In Case 2, we might also have to define the window size correct. After we pass 45k photos through the CNN we get 45k vectors and their corresponding label. Now we can't pass 45k images at one go the LSTM model and create a 45k time step LSTM. We might define lets say 1 second as the window which means we create an LSTM of 30 timsteps(30 is the frame per second ). So the LSTM model is a many to many model with 30 inputs and 30 outputs.

## Batch load the dataset

- Load data
- Train and Test split (Temporal Split)
- Batches of images ( using mini batch SGD )

In [58]:
import os
import numpy as np
import random
from scipy import pi
from itertools import islice
import random
import scipy
import scipy.misc
import cv2


xs = []
ys = []

#pints to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open(r"C:\Users\sweta\OneDrive\Desktop\DL Projects\Self driving cars\Autopilot\driving_dataset\data.txt") as f:
    for line in f:
        xs.append(r"C:\Users\sweta\OneDrive\Desktop\DL Projects\Self driving cars\Autopilot\driving_dataset/" + line.split()[0])
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)

train_xs = xs[:int(len(xs) * 0.8)]
train_ys = ys[:int(len(ys) * 0.8)]
                  
val_xs = xs[-int(len(xs) * 0.2) :]   
val_ys = ys[-int(len(ys) * 0.2) :]
                  
num_train_img = len(train_xs)
num_val_img = len(val_xs)
from PIL import Image
def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        arr = cv2.imread(train_xs[(train_batch_pointer + i) % num_train_img])
        img_road = arr[-150:]
        img_road_resize = cv2.resize(img_road, (200, 66))
        read_image_final = img_road_resize/255.0
        x_out.append(read_image_final)
        
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_img]])
        
    train_batch_pointer += batch_size
    return x_out, y_out     
def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):        
        arr = cv2.imread(val_xs[(val_batch_pointer + i) % num_val_img])
        img_road = arr[-150:]
        img_road_resize = cv2.resize(img_road, (200, 66))
        read_image_final = img_road_resize/255.0
        x_out.append(read_image_final)
        
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_img]])
        
    val_batch_pointer += batch_size
    return x_out, y_out   